In [1]:
# -*- coding: utf-8 -*-
import codecs
import re
import sys
from collections import Counter, defaultdict, OrderedDict

# Preprocessing corpus

In [2]:
STOPWORDS_FILENAME = 'pap/stopwords1.txt'

stopwords = set([line.strip() for line in codecs.open(STOPWORDS_FILENAME, encoding='utf-8').readlines()])

In [ ]:
stopwords

In [3]:
def remove_special_chars(text):
    return re.compile(r'[\W\d]+', re.UNICODE).sub(' ', text)

In [4]:
def remove_whitespace(text):
    return re.compile(r'[\s\n]+', re.UNICODE).sub(' ', text)

In [5]:
def remove_stopwords(words):
    return [word for word in words if word not in stopwords]

In [6]:
def preprocess_corpus(corpus):
    corpus = corpus.strip().lower()
    corpus = remove_special_chars(corpus)
    corpus = remove_whitespace(corpus)
    words = corpus.split()
    return remove_stopwords(words)

In [7]:
text = u'Jesień w 2017 Japonii jest przepiękna.'
for word in preprocess_corpus(text):
    print word

jesień
japonii
przepiękna


In [8]:
CORPUS_PAP = 'pap/pap10000.txt'

In [9]:
with codecs.open(CORPUS_PAP, encoding='utf-8') as f:
    corpus = f.read().replace('\n', '')

In [ ]:
corpus

Getting the corpus with only relevant words.

In [10]:
words = preprocess_corpus(corpus) 
# words

In [11]:
denominator = len(words) #number of words in corpus
# denominator

Count words frequency in corpus

In [12]:
word_frequency = Counter(words)
# word_frequency

In [13]:
word_frequency['rocznicy']

6

Count the frequency of pair words

In [15]:
def count_pairs(words, offset):
    d = Counter()
    for word in words:
        index = words.index(word);
        if(index < len(words) - offset):
            for i in range(index, index+offset):
                pair = (word, words[i+1])
                if(d[(words[i+1], word)] > 0):
                    pair = (words[i+1], word)
                d[pair] += 1
#                 print i, pair
    return d

In [16]:
pairs_frequency = count_pairs(words, 5)

In [ ]:
pairs_frequency

In [17]:
pairs_frequency[('pomnikiem', 'premiera')]

2

Sort the pairs according to their frequency in corpus

In [18]:
pairs_frequency.most_common(20)

[((u'proc', u'uzgodniona'), 340),
 ((u'proc', u'prognoza'), 340),
 ((u'proc', u'pkb'), 340),
 ((u'proc', u'ekonomiczna'), 340),
 ((u'proc', u'r'), 340),
 ((u'powiedzia\u0142', u'rz\u0105du'), 183),
 ((u'powiedzia\u0142', u'potrzeb\u0119'), 183),
 ((u'powiedzia\u0142', u'argumenty'), 183),
 ((u'powiedzia\u0142', u'uzasadniaj\u0105ce'), 183),
 ((u'powiedzia\u0142', u'przyj\u0119cia'), 183),
 ((u'wobec', u'\u015brod\u0119'), 166),
 ((u'aws', u'komisji'), 135),
 ((u'aws', u'ugrupowania'), 135),
 ((u'aws', u'czele'), 135),
 ((u'aws', u'pose\u0142'), 135),
 ((u'aws', u'stan\u0105\u0142'), 135),
 ((u'sa', u'ag'), 130),
 ((u'sa', u'kwa\u015bniewski'), 130),
 ((u'sa', u'chcia\u0142by'), 130),
 ((u'sa', u'deutsche'), 130)]

Find associations's frequency for given word

In [19]:
def find_associations(pairs, word):
    counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            counter[pair[0][1]] = pair[1]
        if (word == pair[0][1]):
            counter[pair[0][0]] = pair[1]
    return counter

In [20]:
find_associations(pairs_frequency, "premiera").most_common(20)

[(u'ba\u0142tyku', 28),
 (u'srebrnego', 28),
 (u'buzka', 28),
 (u'jerzego', 28),
 (u'pier\u015bcienia', 28),
 (u'nadal', 25),
 (u'wschodzie', 19),
 (u'odpowiedzi', 7),
 (u'politycznej', 6),
 (u'odby\u0142o', 6),
 (u'krajowej', 6),
 (u'swym', 5),
 (u'stanis\u0142awa', 5),
 (u'firmie', 3),
 (u'min', 3),
 (u'domaga\u0142o', 2),
 (u'odwo\u0142any', 2),
 (u'poleg\u0142ym', 2),
 (u'dzia\u0142acza', 2),
 (u'potwierdzaj\u0105', 2)]

Find the associantion's strenght for given word

In [21]:
def find_strenght(pairs, word, alpha = 0.66, beta = 0.00002, gamma = 0.00002):
    strenght_counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            association = pair[0][1]
        elif(word == pair[0][1]):
            association = pair[0][0]
        else:
            continue
        
        assoc_freq = word_frequency[association]
        pair_freq = pair[1]
        if (assoc_freq > beta * denominator):
            strenght = pair_freq / pow(assoc_freq, alpha)
        else:
            strenght = pair_freq / (gamma * denominator)
        
        strenght_counter[association] = strenght;
        
    return strenght_counter  

In [22]:
find_strenght(pairs_frequency, 'premier').most_common(10)

[(u'zas\u0142ugi', 16.421932490587253),
 (u'mazowiecki', 14.173060920121547),
 (u'tadeusz', 9.615764101927157),
 (u'rozwoju', 6.085608409038503),
 (u'otrzyma\u0142', 5.330707920630837),
 (u'europejskiej', 3.9536429948886784),
 (u'brytyjski', 2.110777485788888),
 (u'm\xf3g\u0142', 1.93790879445964),
 (u'polityczn\u0105', 1.8389563682479446),
 (u'city', 1.6021397551792442)]

In [23]:
for word in find_strenght(pairs_frequency, 'premier').most_common(10):
     print word[0],word[1]

zasługi 16.4219324906
mazowiecki 14.1730609201
tadeusz 9.61576410193
rozwoju 6.08560840904
otrzymał 5.33070792063
europejskiej 3.95364299489
brytyjski 2.11077748579
mógł 1.93790879446
polityczną 1.83895636825
city 1.60213975518


In [24]:
for word in find_strenght(pairs_frequency, 'ekonomiczna').most_common(10):
     print word[0],word[1]

proc 7.25609285689
pkb 2.18776162395
prognoza 1.45285150163
uzgodniona 1.26575659397
komisję 0.691368825372
europejską 0.41087604671
rząd 0.13872431561
polski 0.0826169935917


In [25]:
for word in find_strenght(pairs_frequency, 'polska').most_common(10):
     print word[0],word[1]

przeciwnym 34.8684360391
kontynuować 34.8684360391
wypadku 28.8385155932
proces 18.2512706362
reform 16.8862198863
ue 3.64831437743
cztery 2.67173052104
polityki 2.56685179513
politykę 2.2598182569
udziału 2.18776162395


In [26]:
for word in find_strenght(pairs_frequency, 'kurs').most_common(10):
     print word[0],word[1]

nfi 12.6890174189
zmienił 7.99903813253
rynku 7.48409030294
wzrósł 7.35020242673
spadł 3.64831437743
sesji 3.21412657691
papierów 3.02754730954
wartościowych 2.98744282935
giełdy 2.9039388321
akcji 2.11938308895


In [27]:
for word in find_strenght(pairs_frequency, 'minister').most_common(10):
     print word[0],word[1]

dziedzictwa 43.668602492
zakrzewski 43.668602492
narodowego 19.1022438311
kultury 11.0695483665
andrzej 6.48170767161
czwartek 4.06830190858
powiedziała 2.67173052104
zmarł 2.32767114913
ministrów 2.2598182569
później 2.18776162395


In [28]:
for word in find_strenght(pairs_frequency, 'rosja').most_common(10):
     print word[0],word[1]

stosuje 21.0
metodami 13.2904442367
zgadzamy 10.1699605114
kwietniu 2.02791895958
problemy 1.93790879446
dialogu 1.72842206343
konfliktu 1.72842206343
międzynarodowa 1.60213975518
kaukazie 1.60213975518
mołdawii 1.45285150163


In [29]:
for word in find_strenght(pairs_frequency, 'niemcy').most_common(10):
    print word[0],word[1]

przyjaciół 2.90570300326
izraela 1.83895636825
nazwał 1.60213975518
najważniejszych 1.31265697437
przekonani 1.26575659397
mówił 1.103948053
jednym 1.05125178647
ehud 1.0
deportowano 1.0
celne 1.0


In [30]:
for word in find_strenght(pairs_frequency, 'polityka').most_common(10):
    print word[0],word[1]

przygotowaniu 8.0
pokojowej 5.06302637588
skarbu 2.76917370155
rewolucji 2.76547530149
środkowej 2.76547530149
europie 1.87624665403
uzasadnieniu 1.26575659397
rolnej 1.26575659397
medialnej 1.26575659397
deputowani 1.0
